In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cv2
from decimal import Decimal
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider
from scipy import interpolate
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.widgets import Slider
import pickle


In [ ]:
#READ PICKLE FILE
# process_data_path = '/home/augustine/lfd_ws/src/skill_transfer/hmpar_former/data_process/process_data/'
raw_data_path  = '/home/augustine/lfd_ws/src/skill_transfer/hmpar_former/data_process/raw_data/'

fileroot='24_08_19/'
filename='Khoa_01'

# Read the raw pickle file
with open(raw_data_path + fileroot + filename + '.pkl', 'rb') as f:
    data = pickle.load(f)

# Print the data
for key, value in data.items():
    print(f"Key: {key}")
    print(f"Shape: {value.shape}")
    # print(f"Shape: {value[0][0]}")
    # print(f"Shape: {value[1][0]}")
    # print(f"Shape: {value[2][0]}")
    # print(f"Shape: {value[3][0]}")
    # print(f"Shape: {value[4][0]}")
    print()

# Trim the data
n = 3
m = 3
data = {key: value[n:-m] if m > 0 else value[n:] for key, value in data.items()}


# Print the shape of the trimmed data to confirm
# for key, value in data.items():
#     print(f"Key: {key}")
#     print(f"Shape: {value[0][0]}")
#     print(f"Shape: {value[1][0]}")
#     print(f"Shape: {value[2][0]}")
#     print(f"Shape: {value[3][0]}")
#     print(f"Shape: {value[4][0]}")
#     print()

In [ ]:

import plotly.graph_objects as go
import numpy as np
from ipywidgets import interact, IntSlider, Layout
from IPython.display import display

def visualize_skeleton_3d(skeleton_data, timestamps):
    # Define the connections between joints for 6 points
    connections = [
        (0, 1), (1, 2),  # Right arm
        (3, 4), (4, 5),  # Left arm
        (2, 3),  # Connection between arms
    ]
    
    # Calculate the overall min and max for each axis
    x_min, y_min, z_min = np.min(skeleton_data, axis=(0, 1))
    x_max, y_max, z_max = np.max(skeleton_data, axis=(0, 1))
    
    # Add some padding to the min and max values
    padding = 0.1  # 10% padding
    x_range = x_max - x_min
    y_range = y_max - y_min
    z_range = z_max - z_min
    x_min -= x_range * padding
    x_max += x_range * padding
    y_min -= y_range * padding
    y_max += y_range * padding
    z_min -= z_range * padding
    z_max += z_range * padding
    
    # Extend axes to negative values
    x_limit = max(abs(x_min), abs(x_max))
    y_limit = max(abs(y_min), abs(y_max))
    z_limit = max(abs(z_min), abs(z_max))
    
    # Create an initial figure widget
    scatter = go.Scatter3d(
        x=skeleton_data[0][:, 0], y=skeleton_data[0][:, 1], z=skeleton_data[0][:, 2],
        mode='markers',
        marker=dict(size=6, color='blue'),
        name='Joints'
    )
    
    lines = []
    for start, end in connections:
        lines.append(
            go.Scatter3d(
                x=[skeleton_data[0][start, 0], skeleton_data[0][end, 0]],
                y=[skeleton_data[0][start, 1], skeleton_data[0][end, 1]],
                z=[skeleton_data[0][start, 2], skeleton_data[0][end, 2]],
                mode='lines',
                line=dict(color='red', width=2),
                name=f'Connection {start}-{end}'
            )
        )
    
    data = [scatter] + lines
    
    layout = go.Layout(
        scene=dict(
            xaxis=dict(title='X', range=[-x_limit, x_limit], autorange=False),
            yaxis=dict(title='Y', range=[-y_limit, y_limit], autorange=False),
            zaxis=dict(title='Z', range=[-z_limit, z_limit], autorange=False),
            aspectmode='cube',  # This ensures the plot is a cube
            camera=dict(
                eye=dict(x=1, y=-0.2, z=-2),  # Camera position: looking along z-axis
                up=dict(x=0, y=1, z=0)     # Up direction
            )
        ),
        title=f"Frame: 0, Timestamp: {timestamps[0]}",
        height=700,
        width=700
    )
    
    fig = go.FigureWidget(data=data, layout=layout)
    
    # Function to update the figure
    def update_frame(frame):
        with fig.batch_update():
            fig.data[0].x = skeleton_data[frame][:, 0]
            fig.data[0].y = skeleton_data[frame][:, 1]
            fig.data[0].z = skeleton_data[frame][:, 2]
            
            for i, (start, end) in enumerate(connections):
                fig.data[i + 1].x = [skeleton_data[frame][start, 0], skeleton_data[frame][end, 0]]
                fig.data[i + 1].y = [skeleton_data[frame][start, 1], skeleton_data[frame][end, 1]]
                fig.data[i + 1].z = [skeleton_data[frame][start, 2], skeleton_data[frame][end, 2]]
            
            fig.layout.title = f"Frame: {frame}, Timestamp: {timestamps[frame]}"
    
    # Display the figure widget
    display(fig)
    
    # Create an interactive slider
    interact(
        update_frame,
        frame=IntSlider(
            min=0, 
            max=len(skeleton_data)-1, 
            step=1, 
            value=0, 
            description='Frame:',
            style={'description_width': 'initial'},
            layout=Layout(width='1250px')
        )
    )

# Assuming interpolated_skeletal_data is your numpy array of shape (frames, 6, 3)
# and skeleton_timestamps is your array of timestamps

visualize_skeleton_3d(data['points'], data['timestamp'])
